In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [2]:
path = '/Users/sheenmccullough/Documents/Kaggle/M5Forecasting/'

calendar = pd.read_csv(path+'calendar.csv')
trainData = pd.read_csv(path+'sales_train_validation.csv')
sellprices = pd.read_csv(path+'sell_prices.csv')

In [3]:
def Convert_to_transaction(df):
    
    '''
    convert our sales data into a daily transacitons set using the pandas melt function
    '''
    list_id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'] 
    d_columns = df.drop(list_id_vars,axis=1)
    df_melted_sales = df.melt(id_vars = list_id_vars, value_vars = d_columns.columns, var_name = 'days', value_name = 'sales') 
    
    
    return df_melted_sales



In [4]:
train_df_melt = Convert_to_transaction(trainData)

In [5]:
#del train_df_melt
del trainData
gc.collect()

144

In [6]:
train_df = pd.merge(train_df_melt,calendar[['d','wday','wm_yr_wk','event_name_1','date','year']],left_on='days',
                    right_on='d')

In [7]:
train = train_df[train_df['date']<='2016-04-01']
validation = train_df[train_df['date']>'2016-04-01']

In [8]:
def autocorrelation(x, n_lags):
    autocorrels = {'lags':[],'autocorrelation':[]}
    for l in range(1,n_lags):
        mean = x.mean()
        normarray = x[:-l]
        lagarray= x[l:]
        numerator = sum([((yt-mean)*(ytk-mean)) for yt,ytk in zip(normarray,lagarray)])
        denominator = sum([(yt-mean)**2 for yt in normarray])
        
        autocorrels['lags'].append(l)
        if denominator!=0:
            val =numerator/denominator
            autocorrels['autocorrelation'].append(numerator/denominator)

            plt.vlines(l,0,(numerator/denominator))
        else:
            
            autocorrels['autocorrelation'].append(0)

            plt.vlines(l,0,(0))
            
         
    plt.plot(autocorrels['lags'],autocorrels['autocorrelation'],'bo')
    plt.hlines(0,0,n_lags+1)
    upperlimit = 2/np.sqrt(len(normarray))
    lowerlimit = -(2/np.sqrt(len(normarray)))
    plt.hlines(lowerlimit,0,n_lags+1,colors='b',linestyles='dashed')
    plt.hlines(upperlimit,0,n_lags+1,colors='b',linestyles='dashed')
    plt.show()


    return autocorrels


ARIMA
     -Autoregressive: When working with a time series and a previous values has a linear relationship with the current value
     
     -Integrated: These are for if you want to predict the difference between pairs of y values as opposed to predicting y directly.
     
     -Moving Average: The difference between the true values of y at previous points in time and our prediction
     
     